In [1]:
import os
import rosbag
from matplotlib import pyplot as plt
import cv2
import numpy as np
from cv_bridge import CvBridge

from geometry_msgs.msg import Twist
from sensor_msgs.msg import Image
#from bebop_msgs.msg import Ardrone3PilotingStateAltitudeChanged

In [2]:
nbFrames = 2

bag = rosbag.Bag('test_2020-06-15-19-36-39.bag')
bridge = CvBridge()

cmdVel = []
optFlow = []
pairs = dict()
pairs["flow"] = []
pairs["cmd"] = []

for topic, msg, t in bag.read_messages(topics=['/bebop/cmd_vel', '/flow']):
    if topic == '/flow':
        optFlow.append(bridge.imgmsg_to_cv2(msg, 'bgr16'))
        
    if topic == '/bebop/cmd_vel':
        cmdVel.append(msg.linear)
        if len(optFlow) >= nbFrames:
            for i in range(nbFrames):
                tmp = np.flip(optFlow[len(optFlow) - 1 + i - nbFrames][...,1:3], axis=-1).astype(np.float32)
                tmp = (tmp - 32768.) / (64.)
                pairs["flow"].append(tmp)
            pairs["cmd"].append(cmdVel[-1])

print("Number of cmds: " + str(len(pairs["cmd"])))
print("Number of flows: " + str(len(pairs["flow"])))

print(pairs["cmd"][201])
print(pairs["flow"][201][...,0])

bag.close()

Number of cmds: 353
Number of flows: 706
x: 0.1
y: 0.0
z: 0.0
[[-0.03125  -0.015625  0.015625 ... -0.265625 -0.265625 -0.265625]
 [-0.03125  -0.015625  0.015625 ... -0.265625 -0.265625 -0.265625]
 [-0.03125   0.        0.015625 ... -0.265625 -0.265625 -0.265625]
 ...
 [-2.765625 -3.03125  -3.296875 ... -1.328125 -1.328125 -1.328125]
 [-2.765625 -3.03125  -3.296875 ... -1.328125 -1.328125 -1.328125]
 [-2.765625 -3.03125  -3.296875 ... -1.328125 -1.328125 -1.328125]]


In [13]:
print(pairs["cmd"][201])
print(optFlow[100][...,1])
print(optFlow[100][...,2])
np.set_printoptions(precision=3)
print(pairs["flow"][201][...,0])
print(pairs["flow"][201][...,1])

x: 0.1
y: 0.0
z: 0.0
[[32768 32768 32768 ... 32768 32768 32768]
 [32768 32768 32768 ... 32768 32768 32768]
 [32768 32768 32768 ... 32768 32768 32768]
 ...
 [32768 32768 32768 ... 32768 32768 32768]
 [32768 32768 32768 ... 32768 32768 32768]
 [32768 32768 32768 ... 32768 32768 32768]]
[[32769 32770 32771 ... 33173 33173 33173]
 [32768 32769 32769 ... 33175 33175 33175]
 [32767 32767 32767 ... 33177 33177 33177]
 ...
 [32225 32222 32219 ... 33400 33400 33400]
 [32225 32222 32219 ... 33400 33400 33400]
 [32225 32222 32219 ... 33400 33400 33400]]
[[-0.031 -0.016  0.016 ... -0.266 -0.266 -0.266]
 [-0.031 -0.016  0.016 ... -0.266 -0.266 -0.266]
 [-0.031  0.     0.016 ... -0.266 -0.266 -0.266]
 ...
 [-2.766 -3.031 -3.297 ... -1.328 -1.328 -1.328]
 [-2.766 -3.031 -3.297 ... -1.328 -1.328 -1.328]
 [-2.766 -3.031 -3.297 ... -1.328 -1.328 -1.328]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 

In [39]:
#print(optFlow[100][...,1])
#print(optFlow[100][...,2])
      
test = np.flip(optFlow[14][...,1:3], axis=-1).astype(np.float32)
#print(test[...,0])
#print(test[...,1])

test = (test - 32768.) / (64.)
#print(test[...,0])
#print(test[...,1])
print(np.amax(test[...,0]))
print(np.amax(test[...,1]))

4.375
0.0


In [32]:
cvImage = cv2.imread('flow_gt.png', -1)
print(cvImage[...,1])
print(cvImage[...,2])

test1 = np.flip(cvImage[...,1:3], axis=-1).astype(np.float32)
test1 = (test1 - 32768.) / (64.)

print(test1[...,0])
print(test1[...,1])
print(np.amax(test1[...,1]))

[[32768 32768 32768 ... 32768 32768 32768]
 [32768 32768 32768 ... 32768 32768 32768]
 [32768 32768 32768 ... 32768 32768 32768]
 ...
 [32768 32768 36569 ... 32768 32768 32768]
 [32768 32768 32768 ... 32768 32768 32768]
 [32768 32768 32768 ... 32768 32768 32768]]
[[32768 32768 32768 ... 32768 32768 32768]
 [32768 32768 32768 ... 32768 32768 32768]
 [32768 32768 32768 ... 32768 32768 32768]
 ...
 [32768 32768 20540 ... 32768 32768 32768]
 [32768 32768 32768 ... 32768 32768 32768]
 [32768 32768 32768 ... 32768 32768 32768]]
[[-0.391 -0.328 -0.25  ...  0.062  0.062  0.062]
 [-0.359 -0.312 -0.25  ...  0.094  0.094  0.094]
 [-0.328 -0.297 -0.25  ...  0.141  0.141  0.141]
 ...
 [ 1.344  1.438  1.547 ... -0.688 -0.688 -0.688]
 [ 1.344  1.438  1.547 ... -0.688 -0.688 -0.688]
 [ 1.344  1.438  1.547 ... -0.688 -0.688 -0.688]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
75.546875
